In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://www.airlinequality.com/robots.txt"
response = requests.get(url)

if response.status_code == 200:
    print("Robots.txt is available.")
    print(response.text)
else:
    print("Robots.txt not found. HTTP Status Code:", response.status_code)

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/{}/page/{}"
airline = "turkish-airlines"
num_pages = 2

category_mapping = {
    "Aircraft": "Aircraft",
    "Type Of Traveller": "Type Of Traveller",
    "Seat Type": "Seat Type",
    "Route": "Route",
    "Date Flown": "Date Flown",
    "Seat Comfort": "Seat Comfort",
    "Cabin Staff Service": "Cabin Staff Service",
    "Food & Beverages": "Food & Beverages",
    "Inflight Entertainment": "Inflight Entertainment",
    "Ground Service": "Ground Service",
    "Wifi & Connectivity": "Wifi & Connectivity",
    "Value For Money": "Value For Money",
    "Recommended": "Recommended"
}
all_reviews_data = []

for page in range(1, num_pages + 1):
    url = base_url.format(airline, page)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        tables = soup.find_all("table")

        for table in tables[1:]:
            review_data = {}

            for row in table.find_all("tr"):
                columns = row.find_all("td")

                if len(columns) == 2:
                    category = columns[0].text.strip()
                    span_fill = columns[1].find_all("span", class_="fill")
                    value = span_fill[-1].text.strip() if span_fill else columns[1].text.strip()

                    if category in category_mapping:
                        review_data[category_mapping[category]] = value

            all_reviews_data.append(review_data)

    else:
        print(f"HTTP Response Code for page {page}: {response.status_code}")

df_reviews = pd.DataFrame(all_reviews_data)

df_reviews = df_reviews.reindex(columns=[
    "Aircraft", "Type Of Traveller", "Seat Type", "Route", "Date Flown",
    "Seat Comfort", "Cabin Staff Service", "Food & Beverages",
    "Inflight Entertainment", "Ground Service", "Wifi & Connectivity",
    "Value For Money", "Recommended"
])

print(df_reviews)